In [292]:
#importing libraries
import spacy
from spacy.vocab import Vocab
import numpy
from numpy import array
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.models import load_model
import unicodecsv
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA
from spacy.tokens import Doc
from spacy.matcher import Matcher
import re
import pickle

In [356]:
#reading processed data
data = open(r'C:\Users\jsedl\Downloads\blogtext.csv', encoding='utf-8').read()[:10000]
data
#data=unicodecsv.reader(data, encoding='utf-8', delimeter = ';')
#data = "I think that students would benefit from learning at home,because they wont have to change and get up early in the morning to shower and do there hair. taking only classes helps them because at there house they'll be pay more attention. they will be comfortable at home. The hardest part of school is getting ready. you wake up go brush your teeth and go to your closet and look at your cloths. after you think you picked a outfit u go look in the mirror and youll either not like it or you look and see a stain. Then you'll have to change. with the online classes you can wear anything and stay home and you wont need to stress about what to wear. most students usually take showers before school. they either take it before they sleep or when they wake up. some students do both to smell good. that causes them do miss the bus and effects on there lesson time cause they come late to school. when u have online classes u wont need to miss lessons cause you can get everything set up and go take a shower and when u get out your ready to go. when your home your comfortable and you pay attention. it gives then an advantage to be smarter and even pass there classmates on class work. public schools are difficult even if you try. some teacher dont know how to teach it in then way that students understand it. that causes students to fail and they may repeat the class. "



'id,gender,age,topic,sign,date,text\n2059027,male,15,Student,Leo,"14,May,2004","           Info has been found (+/- 100 pages, and 4.5 MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html.         "\n2059027,male,15,Student,Leo,"13,May,2004","           These are the team members:   Drewes van der Laag           urlLink mail  Ruiyu Xie                     urlLink mail  Bryan Aaldering (me)          urlLink mail          "\n2059027,male,15,Student,Leo,"12,May,2004","           In het kader van kernfusie op aarde:  MAAK JE EIGEN WATERSTOFBOM   How to build an H-Bomb From: ascott@tartarus.uwa.edu.au (Andrew Scott) Newsgroups: rec.humor Subject: How To Build An H-Bomb (humorous!) Date: 7 Feb 1994 07:41:14 GMT Organization: The University of Western Australia  Original file dated 12th November 1990. Seemed to be a transcript of a \'Seven Days\' article. Poorly formatted and corrupted. I have added the text between \'examine under a microscope\' and \'

In [102]:
# attempt to read with pandas...may need if cannot fix w other
# myFile=open(r'C:\Users\jsedl\Downloads\blogtext.csv')
# data=pd.read_csv(myFile, encoding = 'utf-8', quotechar='"' ,delimiter = ';', nrows=1000000)
# type(data)
# data

In [103]:
#function for preparing text data into sequences for training 
def data_sequencing(data):   
    # integer encode sequences of words
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([data])
    with open('tokenizer.pkl', 'wb') as f: # Save the tokeniser by pickling it
        pickle.dump(tokenizer, f)

    encoded = tokenizer.texts_to_sequences([data])[0]
    # retrieve vocabulary size
    vocab_size = len(tokenizer.word_index) + 1
    print('Vocabulary Size: %d' % vocab_size)
    
    # create line-based sequences
    sequences = list()
    rev_sequences = list()
    for line in data.split('.'):
        encoded = tokenizer.texts_to_sequences([line])[0]
        rev_encoded = encoded[::-1]
        for i in range(1, len(encoded)):
            sequence = encoded[:i+1]
            rev_sequence = rev_encoded[:i+1]
            sequences.append(sequence)
            rev_sequences.append(rev_sequence)
    print('Total Sequences: %d' % len(sequences))
    
    
    #find max sequence length 
    max_length = max([len(seq) for seq in sequences])
    with open('max_length.pkl', 'wb') as f: # Save max_length by pickling it
        pickle.dump(max_length, f)
    print('Max Sequence Length: %d' % max_length)

    # pad sequences and create the forward sequence
    sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
    # split into input and output elements
    sequences = array(sequences)
    X, y = sequences[:,:-1],sequences[:,-1]
    
    #pad sequences and create the reverse sequencing
    rev_sequences = pad_sequences(rev_sequences, maxlen=max_length, padding='pre')
    # split into input and output elements
    rev_sequences = array(rev_sequences)
    rev_X, rev_y = rev_sequences[:,:-1],rev_sequences[:,-1]

    return X,y,rev_X,rev_y,max_length,vocab_size

In [104]:
#returning forward and reverse sequences along with max sequence 
#length from the data 

X,y,rev_X,rev_y,max_length,vocab_size = data_sequencing(data)

Vocabulary Size: 740
Total Sequences: 1599
Max Sequence Length: 56


In [105]:
# define forward sequence model
model = Sequential()
model.add(Embedding(vocab_size,100, input_length=max_length-1))
#model.add(LSTM(100))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 55, 100)           74000     
                                                                 
 bidirectional_12 (Bidirecti  (None, 200)              160800    
 onal)                                                           
                                                                 
 dense_11 (Dense)            (None, 740)               148740    
                                                                 
Total params: 383,540
Trainable params: 383,540
Non-trainable params: 0
_________________________________________________________________
None


In [106]:
# define reverse model
rev_model = Sequential()
rev_model.add(Embedding(vocab_size, 100, input_length=max_length-1))
#rev_model.add(LSTM(100))
rev_model.add(Bidirectional(LSTM(100)))
rev_model.add(Dense(vocab_size, activation='softmax'))
print(rev_model.summary())

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 55, 100)           74000     
                                                                 
 bidirectional_13 (Bidirecti  (None, 200)              160800    
 onal)                                                           
                                                                 
 dense_12 (Dense)            (None, 740)               148740    
                                                                 
Total params: 383,540
Trainable params: 383,540
Non-trainable params: 0
_________________________________________________________________
None


In [107]:
# compile forward sequence network
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y,batch_size=100, epochs=100, verbose=2)
# save the model to file
model.save('model.h5')

Epoch 1/100
16/16 - 15s - loss: 6.5485 - accuracy: 0.0225 - 15s/epoch - 967ms/step
Epoch 2/100
16/16 - 13s - loss: 6.0941 - accuracy: 0.0275 - 13s/epoch - 782ms/step
Epoch 3/100
16/16 - 13s - loss: 5.9467 - accuracy: 0.0356 - 13s/epoch - 814ms/step
Epoch 4/100
16/16 - 25s - loss: 5.9154 - accuracy: 0.0356 - 25s/epoch - 2s/step
Epoch 5/100
16/16 - 26s - loss: 5.9097 - accuracy: 0.0281 - 26s/epoch - 2s/step
Epoch 6/100
16/16 - 28s - loss: 5.8967 - accuracy: 0.0356 - 28s/epoch - 2s/step
Epoch 7/100
16/16 - 31s - loss: 5.8852 - accuracy: 0.0356 - 31s/epoch - 2s/step
Epoch 8/100
16/16 - 30s - loss: 5.8683 - accuracy: 0.0394 - 30s/epoch - 2s/step
Epoch 9/100
16/16 - 30s - loss: 5.8389 - accuracy: 0.0356 - 30s/epoch - 2s/step
Epoch 10/100
16/16 - 30s - loss: 5.7867 - accuracy: 0.0482 - 30s/epoch - 2s/step
Epoch 11/100
16/16 - 30s - loss: 5.7152 - accuracy: 0.0500 - 30s/epoch - 2s/step
Epoch 12/100
16/16 - 30s - loss: 5.6128 - accuracy: 0.0557 - 30s/epoch - 2s/step
Epoch 13/100
16/16 - 30s - l

In [108]:
# compile reverse sequence network
rev_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
rev_model.fit(rev_X, rev_y,batch_size=100, epochs=100, verbose=2)
# save the model to file
rev_model.save('rev_model.h5')

Epoch 1/100
16/16 - 11s - loss: 6.5387 - accuracy: 0.0275 - 11s/epoch - 680ms/step
Epoch 2/100
16/16 - 8s - loss: 6.0607 - accuracy: 0.0344 - 8s/epoch - 488ms/step
Epoch 3/100
16/16 - 8s - loss: 5.9114 - accuracy: 0.0344 - 8s/epoch - 493ms/step
Epoch 4/100
16/16 - 8s - loss: 5.8846 - accuracy: 0.0381 - 8s/epoch - 493ms/step
Epoch 5/100
16/16 - 8s - loss: 5.8680 - accuracy: 0.0375 - 8s/epoch - 490ms/step
Epoch 6/100
16/16 - 8s - loss: 5.8488 - accuracy: 0.0400 - 8s/epoch - 489ms/step
Epoch 7/100
16/16 - 8s - loss: 5.8125 - accuracy: 0.0388 - 8s/epoch - 495ms/step
Epoch 8/100
16/16 - 8s - loss: 5.7752 - accuracy: 0.0500 - 8s/epoch - 492ms/step
Epoch 9/100
16/16 - 8s - loss: 5.7206 - accuracy: 0.0438 - 8s/epoch - 492ms/step
Epoch 10/100
16/16 - 8s - loss: 5.6321 - accuracy: 0.0475 - 8s/epoch - 492ms/step
Epoch 11/100
16/16 - 8s - loss: 5.5474 - accuracy: 0.0519 - 8s/epoch - 495ms/step
Epoch 12/100
16/16 - 8s - loss: 5.4513 - accuracy: 0.0550 - 8s/epoch - 496ms/step
Epoch 13/100
16/16 - 8s

In [109]:
# generate a sequence using a language model
def generate_seq(model, tokenizer, max_length, seed_text):
    if seed_text == "":
        return ""
    else:
        in_text = seed_text
        n_words = 1
        n_preds = 5 #number of words to predict for the seed text
        pred_words = ""
        # generate a fixed number of words
        for _ in range(n_words):
            # encode the text as integer
            encoded = tokenizer.texts_to_sequences([in_text])[0]
            # pre-pad sequences to a fixed length
            encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
            # predict probabilities for each word
            proba = model.predict(encoded, verbose=0).flatten()
            #take the n_preds highest probability classes 
            yhat = numpy.argsort(-proba)[:n_preds] 
            # map predicted words index to word
            out_word = ''

            for _ in range(n_preds):
                for word, index in tokenizer.word_index.items():
                    if index == yhat[_] and word not in stopwords: # CHECK THIS 
                        out_word = word
                        pred_words += ' ' + out_word
                        #print(out_word)
                        break


        return pred_words

In [110]:
# load the model
model = load_model('model.h5')
rev_model = load_model('rev_model.h5')

#load tokeniser and max_length
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
    
with open('max_length.pkl', 'rb') as f:
    max_length = pickle.load(f)
    

#load spacy GloVe Model
nlp = spacy.load('en_core_web_md')

#loading stopwords to improve relevant word predictions    
stopwords= nlp.Defaults.stop_words



In [351]:
#Find and set embeddings for OOV words
def set_embedding_for_oov(doc):
    #checking for oov words and adding embedding
    print(type(doc))
    j=0 #for keeping track of index to replace
    changed_string=""
    doc2=doc
    for token in doc:
        changed_string=changed_string+str(token.text_with_ws)
        if token.is_oov == True:

            
            
            before_text = doc[:token.i].text
            after_text = str(array(doc)[:token.i:-1]).replace('[','').replace(']','')

            pred_before = generate_seq(model, tokenizer, max_length-1, before_text).split()
            pred_after = generate_seq(rev_model, tokenizer, max_length-1, after_text).split()
            
            embedding = numpy.zeros((300,))

            i=len(before_text)
            print('Words predicted from forward sequence model:')
            for word in pred_before:
                print(word)
                embedding += i*nlp.vocab.get_vector(word)
                i= i*.5
            i=len(after_text)
            print('Words predicted from reverse sequence model:')
            for word in pred_after:
                print(word)
                embedding += i*nlp.vocab.get_vector(word)
                i= i*.5
            nlp.vocab.set_vector(token.text, embedding)  #CHECK: So we don't include the shitty words?

            
        
           
             
            #set vector back to zero?
            #nlp.vocab.set_vector(token.text, 0)

##########################################################################################################
            
            #Locate similar words
            by_similarity = sorted(token.vocab, key=lambda w: token.similarity(w), reverse=True)
            #find the closest word to the token to be a replacement
            replace=[w.orth_ for w in by_similarity[:3]][2]
            #cast to string
            token=str(token)
         
            
            
            changed_string=changed_string.replace(token, replace)

            #create new doc with replaced word
            doc2=nlp.make_doc(changed_string)
            
            j=j+1

        doc2=nlp.make_doc(changed_string)

    #assign original document to new document and return original document
    doc=doc2    
    return changed_string  #return doc if you want document returned

In [354]:

doc = nlp("I think that student would benefit from learning attttttt home,because they wont have to change and get up early in the morning to shower and do there hair. taking only classes helps them because at there house they'll be pay more attention. they will be comfortab at home. The hardest part of school is getting ready. you wake up go brush your teeth and go to your closet and look at your cloths. after you think you picked a outfit u go look in the mirror and youll either not like it or you look and see a stain. Then you'll have to change. with the online classes you can wear anything and stay home and you wont need to stress about what to wear. most students usually take showers before school. they either take it before they sleep or when they wake up. some students do both to smell good. that causes them do miss the bus and effects on there lesson time cause they come late to school. when u have online classes u wont need to miss lessons cause you can get everything set up and go take a shower and when u get out your ready to go. when your home your comfortable and you pay attention. it gives then an advantage to be smarter and even pass there classmates on class work. public schools are difficult even if you try. some teacher dont know how to teach it in then way that students understand it. that causes students to fail and they may repeat the class. "
)
#print(nlp.vocab.get_vector('studen'))
doc=set_embedding_for_oov(doc)
#print(nlp.vocab.get_vector('studen'))
doc



<class 'spacy.tokens.doc.Doc'>
<class 'str'>


In [360]:
index=0
arr= ["heldifhadkjf Charlie, you are nice", "hello lkadhfasldkjfhlkj, you are nice", "hello joe, you are stupid" ]
for essay in arr:

    doc=nlp(essay)
    fixed=set_embedding_for_oov(doc)
    arr[index]=fixed
    index=index+1
print(arr)

<class 'spacy.tokens.doc.Doc'>
Words predicted from forward sequence model:
Words predicted from reverse sequence model:
gulf
worse
<class 'spacy.tokens.doc.Doc'>


C:\Users\jsedl\AppData\Local\Temp\ipykernel_21384\1166202396.py:46: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  by_similarity = sorted(token.vocab, key=lambda w: token.similarity(w), reverse=True)


Words predicted from forward sequence model:
you've
better
Words predicted from reverse sequence model:
gulf
worse
<class 'spacy.tokens.doc.Doc'>
['Miss. Charlie, you are nice', 'hello hellllllllllll, you are nice', 'hello joe, you are stupid']


In [341]:
lis=most_similar(nlp('comfortab'))[0]
print(type(lis))





<class 'str'>


C:\Users\jsedl\AppData\Local\Temp\ipykernel_21384\1143453434.py:3: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  by_similarity = sorted(word.vocab, key=lambda w: word.similarity(w), reverse=True)


In [251]:
#function to find most similar words
def most_similar(word):
    by_similarity = sorted(word.vocab, key=lambda w: word.similarity(w), reverse=True)
    return [w.orth_ for w in by_similarity[:1]] #change :x for x amoint of similar words